In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [6]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.0.1
3.1


In [7]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [10]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [11]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [12]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [13]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x121c19e40>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Damage', 'A.4_Dike Investment Costs'), functio

In [14]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [01:34<00:00,  2.12it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [15]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,64,205.365210,1.0,0.288224,257.548185,1.5,0.393544,318.207520,10.0,0.783991,...,6,7,4,9,6,1,1,4,0,dikesnet
1,37,68.724598,1.5,0.497429,335.622581,10.0,0.939751,79.277629,1.0,0.626310,...,6,7,4,9,6,1,1,5,0,dikesnet
2,125,96.076736,1.5,0.597215,303.722036,10.0,0.793522,303.501131,1.0,0.835825,...,6,7,4,9,6,1,1,6,0,dikesnet
3,130,317.261748,1.0,0.250318,214.109489,1.0,0.131701,298.309262,10.0,0.609279,...,6,7,4,9,6,1,1,7,0,dikesnet
4,69,287.512843,1.5,0.193586,93.745051,1.5,0.851633,271.343696,10.0,0.371513,...,6,7,4,9,6,1,1,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,75,266.625886,10.0,0.985194,48.851730,10.0,0.457623,84.008456,1.0,0.425501,...,0,8,0,2,2,9,3,49,3,dikesnet
196,85,118.316062,1.0,0.762466,310.748269,1.5,0.572978,245.644430,10.0,0.451376,...,0,8,0,2,2,9,3,50,3,dikesnet
197,15,41.462346,1.0,0.430579,193.714610,1.5,0.264062,282.140231,1.5,0.840104,...,0,8,0,2,2,9,3,51,3,dikesnet
198,81,30.082128,1.0,0.215345,139.556062,10.0,0.709997,218.246392,1.5,0.184443,...,0,8,0,2,2,9,3,52,3,dikesnet


In [16]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,1.267450e+08,0.0,1.585614e+08,0.000275,4.587651e+07,0.000000,4.503928e+07,0.000000,1.490532e+08,0.000000,1.382500e+09,20.318870
1,1.267450e+08,0.0,1.579233e+08,0.000000,4.587651e+07,0.000000,4.503928e+07,0.000000,1.490532e+08,0.000000,1.382500e+09,0.000000
2,1.267450e+08,0.0,1.579233e+08,0.000000,4.587651e+07,0.000000,4.503928e+07,0.000000,1.490532e+08,0.000000,1.382500e+09,0.000000
3,1.267450e+08,0.0,1.633207e+08,0.002232,4.587651e+07,0.000000,4.503928e+07,0.000000,1.490532e+08,0.000000,1.382500e+09,174.039199
4,1.267450e+08,0.0,1.579233e+08,0.000000,4.900186e+07,0.001886,4.503928e+07,0.000000,1.490532e+08,0.000000,1.382500e+09,62.217504
...,...,...,...,...,...,...,...,...,...,...,...,...
195,3.588340e+08,0.0,2.119675e+08,0.000000,1.026176e+08,0.000000,1.701990e+07,0.000074,2.149411e+08,0.011939,4.543000e+08,7342.809555
196,3.588340e+08,0.0,2.119675e+08,0.000000,1.026176e+08,0.000000,2.120159e+07,0.000244,1.178279e+08,0.000000,4.543000e+08,258.297669
197,3.588340e+08,0.0,2.119675e+08,0.000000,1.026176e+08,0.000000,3.305454e+07,0.001141,1.178279e+08,0.000000,4.543000e+08,1546.207117
198,3.588340e+08,0.0,2.119675e+08,0.000000,1.101651e+08,0.001611,2.311652e+07,0.000419,1.178279e+08,0.000000,4.543000e+08,795.604471


In [17]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [18]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [02:04<00:00,  2.41it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [19]:
experiments, outcomes = results

In [20]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,1.156661e+07,0.011501,7.888630e+07,0.144846,5.659730e+06,0.002992,4.194345e+07,0.038655,253800000.0,0.0
1,5.397251e+07,0.000000,5.671310e+08,0.472584,0.000000e+00,0.000000,0.000000e+00,0.000000,6.903547e+08,0.539844,253800000.0,0.0
2,5.397251e+07,0.000000,3.731337e+07,0.040572,7.060792e+07,0.143227,1.229056e+08,0.060313,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,2.162987e+08,0.196223,1.383529e+09,2.327305,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,1.953286e+07,0.020430,7.534851e+07,0.146153,6.591179e+05,0.000397,3.510215e+08,0.334829,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.000000e+00,0.000000,4.356018e+07,0.039197,1.604427e+08,0.217699,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
296,0.000000e+00,0.000000,1.787799e+08,0.153130,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,1.440205e+09,1.022319,0.000000e+00,0.000000,2.879840e+07,0.000000,7.500804e+06,0.003791,0.000000e+00,0.000000,369700000.0,0.0
298,1.169811e+09,1.024516,0.000000e+00,0.000000,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
